In [1]:
import torch
import pandas as pd
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    pipeline,
)
from torch.utils.data import DataLoader
from langchain.prompts import PromptTemplate

from datasets import Dataset


pd.options.display.float_format = "{:.2f}".format
pd.set_option("max_colwidth", 1200)
pd.set_option("display.width", 1000)
# pd.set_option("display.max_rows", 100)

pd.set_option("future.no_silent_downcasting", True)

/home/harddisk/projects/test_institut/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Предобработка

In [2]:
df = pd.read_csv("./data/data.csv")
df.shape

(376007, 10)

In [3]:
df.to_csv("./data/data.csv", index=False)

In [4]:
df.sample(5)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
188056,c71c7684cd1ebeb275d00c24a640a604,2008046,Академический бакалавр,БЛГ-20-9,Лингвистика,2023 - 2024,I полугодие,"Практический курс испанского языка, уровень В1++",NaN,NaN
73164,19848fb18f08c12a3018334a96afbc01,2109522,Академический бакалавр,БПМ-21-2,Прикладная математика,2022 - 2023,II полугодие,Теория систем автоматического управления,Удовлетворительно,Удовлетворительно
194446,9f8ef330becd9f1eec5a19c421bf8b4e,2204059,Бакалавр,БЭК-22-4,Экономика,2023 - 2024,II полугодие,Деньги. Кредит. Банки,NaN,NaN
372729,daf673772f0e8510d14d4c847f7997e2,2201960,Специалист,СГД-22-2,Горное дело,2022 - 2023,I полугодие,Информатика,Отлично,Отлично
308006,392988a7e7b5f75658ee2f3b541d18ab,2212270,Магистр,ММТ-22-7,Металлургия,2023 - 2024,II полугодие,Преддипломная практика,NaN,NaN


In [5]:
df.dtypes

hash                         object
Номер ЛД                      int64
Уровень подготовки           object
Учебная группа               object
Специальность/направление    object
Учебный год                  object
Полугодие                    object
Дисциплина                   object
Оценка (без пересдач)        object
Оценка (успеваемость)        object
dtype: object

In [6]:
df.isnull().sum()

hash                              0
Номер ЛД                          0
Уровень подготовки                0
Учебная группа                    0
Специальность/направление         0
Учебный год                       0
Полугодие                         0
Дисциплина                        0
Оценка (без пересдач)        187003
Оценка (успеваемость)        179485
dtype: int64

In [7]:
df.columns = [
    "hash",
    "num_ld",
    "level",
    "group",
    "spec",
    "year",
    "half_year",
    "dis",
    "evaluation_1",
    "evaluation_2",
]
df.columns

Index(['hash', 'num_ld', 'level', 'group', 'spec', 'year', 'half_year', 'dis', 'evaluation_1', 'evaluation_2'], dtype='object')

In [8]:
df.duplicated().sum()

0

Проверка уникальности hash

In [9]:
uniq_hash = df.groupby("hash")["num_ld"].nunique()
not_uniq_hash = uniq_hash[uniq_hash > 1]
print(len(not_uniq_hash))
not_uniq_hash

13


hash
0c2cd8750952648f9dc50f858b5d0bba    2
1e9a1773365bc3cc7ff413275250fc92    2
24e72ecc671f6a23b947178c0fb53c20    2
2a4f75b7c555f16b83a17d7d4b4e94a9    2
2f41ab589489046803d432b59e8e6331    2
5eb84149f57391b4d71b55703e3bf496    2
7a64d6504af97cbaccf38a410b532e6b    2
ad5b37dae95b7b05b6fcebaa98a69490    2
b8fd11f0f253b46399a3dd0a4aba7e42    2
bc2b18ddce3cb9fe20fb47af8d0390cf    2
c38a2fd37e008e520c954bef41b98d65    2
ce1ec83cdca108d3064d2b7dfe7bcdde    2
d88cacfae7cec50f00182bf55240f97a    2
Name: num_ld, dtype: int64

In [10]:
two_level = df[df.hash.isin(not_uniq_hash.index)].groupby("hash")["level"].nunique()
two_level = two_level[two_level > 1]

In [11]:
df.loc[df.hash.isin(two_level.index), ["hash", "num_ld", "level"]].drop_duplicates()

,hash,num_ld,level
46229,b8fd11f0f253b46399a3dd0a4aba7e42,2210020,Бакалавр
46259,b8fd11f0f253b46399a3dd0a4aba7e42,2310815,Магистр
51404,ce1ec83cdca108d3064d2b7dfe7bcdde,2212595,Магистр
51426,ce1ec83cdca108d3064d2b7dfe7bcdde,2306102,Бакалавр
118635,c38a2fd37e008e520c954bef41b98d65,1911506,Специалист
118691,c38a2fd37e008e520c954bef41b98d65,2010258,Академический бакалавр
119636,7a64d6504af97cbaccf38a410b532e6b,1901003,Академический бакалавр
119706,7a64d6504af97cbaccf38a410b532e6b,2207879,Бакалавр
129041,ad5b37dae95b7b05b6fcebaa98a69490,2008212,Академический бакалавр
129115,ad5b37dae95b7b05b6fcebaa98a69490,2305284,Магистр


Можно было бы подумать, что на новом уровне высшего образования заводится новое личное дело, но для одного и того же уровня есть несколько номеров личных дел для одного ид студента, поэтому предпологаю, что это ошибка в поставке данных и такие записи необходимо удалить

In [12]:
print(df.shape[0])
df = df[~df.hash.isin(not_uniq_hash.index)]
df.shape[0]

376007


375044

In [13]:
# удалим num_ld за ненадобностью
df = df.drop("num_ld", axis=1)

In [14]:
df.isnull().sum()

hash                 0
level                0
group                0
spec                 0
year                 0
half_year            0
dis                  0
evaluation_1    186598
evaluation_2    179043
dtype: int64

In [15]:
df["level"].unique()

array(['Академический бакалавр', 'Бакалавр', 'Магистр', 'Специалист'],
      dtype=object)

In [16]:
df["level"] = df["level"].replace("Академический бакалавр", "Бакалавр")
df.level.unique()

array(['Бакалавр', 'Магистр', 'Специалист'], dtype=object)

In [17]:
df.group.nunique()

489

In [18]:
df.group.value_counts()

group
БЛГ-20-8                   3167
БЛГ-20-14                  3161
БЛГ-20-5                   2737
БЛГ-20-10                  2736
БЛГ-20-2                   2731
                           ... 
ММТ-22-3_Сиднев А.В.         15
ММТ-22-3_Гуреев Н.М.         15
ММТ-22-3_Видманова А.Н.      15
ММТ-22-3_Тхор М.Ю.           15
МНТМ-23-2А                   14
Name: count, Length: 489, dtype: int64

In [19]:
def change_group(X):
    if "_" in X:
        return X.split("_")[0]
    else:
        return X

In [20]:
df["group"] = df["group"].apply(lambda X: change_group(X))

In [21]:
df.group.nunique()

465

In [22]:
df.year.value_counts()

year
2023 - 2024    157695
2022 - 2023    113668
2021 - 2022     67414
2020 - 2021     28116
2019 - 2020      5512
2018 - 2019      2299
2024 - 2025       282
2025 - 2026        37
2017 - 2018        15
2026 - 2027         5
2027 - 2028         1
Name: count, dtype: int64

In [23]:
df_search = df[["year", "evaluation_2"]]
df_search["evaluation_2"] = df_search["evaluation_2"].isnull()
df_search.value_counts().sort_index()

/tmp/ipykernel_2811050/3089082234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_search["evaluation_2"] = df_search["evaluation_2"].isnull()


year         evaluation_2
2017 - 2018  False               14
             True                 1
2018 - 2019  False             2296
             True                 3
2019 - 2020  False             5433
             True                79
2020 - 2021  False            27210
             True               906
2021 - 2022  False            52132
             True             15282
2022 - 2023  False            93293
             True             20375
2023 - 2024  False            15365
             True            142330
2024 - 2025  False              217
             True                65
2025 - 2026  False               35
             True                 2
2026 - 2027  False                5
2027 - 2028  False                1
Name: count, dtype: int64

Очень много неуд в 2023-2024, а далее учебные года еще не начались

In [24]:
df = df[~df.year.isin(["2024 - 2025", "2025 - 2026", "2026 - 2027", "2027 - 2028"])]

In [25]:
df[df.year == "2023 - 2024"].evaluation_2.unique()

array([nan, 'зачтено', 'Отлично', 'Удовлетворительно', 'Хорошо', 'Неявка',
       'не зачтено', 'Неудовлетворительно'], dtype=object)

In [26]:
df_search = df[df.year == "2023 - 2024"]
df_search.evaluation_2 = df_search.evaluation_2.fillna("Неудовлетворительно")
count_hash = df_search.groupby("hash")["evaluation_2"].agg("count")
count_hash_bad = (
    df_search[df_search["evaluation_2"] == "Неудовлетворительно"]
    .groupby("hash")["evaluation_2"]
    .agg("count")
)

/tmp/ipykernel_2811050/4160421311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_search.evaluation_2 = df_search.evaluation_2.fillna("Неудовлетворительно")


In [27]:
count_hash = pd.DataFrame(count_hash).join(count_hash_bad, rsuffix="_bad")
count_hash

,evaluation_2,evaluation_2_bad
hash,,
000006af6e40c8234a5af27896b7bba5,17,17.00
00083ac3c8aecc3a1cf66029173e56fa,46,46.00
001912a530f78ef9f5168bd00b75ff71,11,10.00
002ce0ab2ee8101b0c19995bdf2b945d,14,13.00
002ed2297ad196e3b8a7e668f32d125b,46,46.00
...,...,...
ffd6328bf93225b17baf0827acd35fd5,17,17.00
ffd8b521d53584075cf239ebabdc46a0,13,13.00
ffe0df6ab7e1747a1a846fb8318f9130,17,16.00


In [28]:
(count_hash["evaluation_2"] == count_hash["evaluation_2_bad"]).sum()

2848

In [29]:
count_hash[count_hash["evaluation_2"] == count_hash["evaluation_2_bad"]][
    "evaluation_2"
].sum()

58085

За 2023-2024 год студентами получено порядка 142330 неуд из них 2848 студентами получено 58085. Даже если удалить только исключительно двочников, то все равно неуд останется слишком много. Поэтому пологаю, что данные за 2023-2024 учебный год недостоверны и подлежат удалению.

In [30]:
df = df[~df.year.isin(["2023 - 2024"])]

In [31]:
df.isnull().sum()

hash                0
level               0
group               0
spec                0
year                0
half_year           0
dis                 0
evaluation_1    44084
evaluation_2    36646
dtype: int64

In [32]:
df.year.value_counts()

year
2022 - 2023    113668
2021 - 2022     67414
2020 - 2021     28116
2019 - 2020      5512
2018 - 2019      2299
2017 - 2018        15
Name: count, dtype: int64

In [33]:
df.half_year.unique()

array(['I полугодие', 'II полугодие'], dtype=object)

In [34]:
df.evaluation_1.unique()

array(['Хорошо', 'Удовлетворительно', 'зачтено', nan, 'Отлично',
       'Неудовлетворительно', 'Неявка', 'не зачтено',
       'Неявка по ув.причине', 'Не допущен'], dtype=object)

Удалим неявки по уважительной причине, так как сами по себе такие случаю не характеризуют успеваемость студента

In [35]:
df = df[df.evaluation_1 != "Неявка по ув.причине"]
df.shape[0]

216685

In [36]:
df["spec"].value_counts()

spec
Лингвистика                                                   47009
Горное дело                                                   32061
Прикладная информатика                                        19870
Информатика и вычислительная техника                          19328
Менеджмент                                                    13321
Бизнес-информатика                                            10514
Экономика                                                     10177
Прикладная математика                                          9943
Информационные системы и технологии                            9528
Металлургия                                                    7843
Материаловедение и технологии материалов                       7357
Технологические машины и оборудование                          6024
Электроника и наноэлектроника                                  5687
Физические процессы горного или нефтегазового производства     4458
Физика                                     

In [37]:
df["dis"].nunique()

1161

In [38]:
df[["year", "half_year"]].drop_duplicates().groupby("year")["half_year"].nunique()

year
2017 - 2018    1
2018 - 2019    2
2019 - 2020    2
2020 - 2021    2
2021 - 2022    2
2022 - 2023    2
Name: half_year, dtype: int64

## Формирование признаков для модели

In [39]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map={"": 0},
    # load_in_8bit=True,
)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.24s/it]


In [40]:
# generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
# generation_config.max_new_tokens = 1000
# generation_config.temperature = 0.001
# generation_config.top_p = 0.95
# generation_config.do_sample = True
# generation_config.repetition_penalty = 1.15


text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    batch_size=16,
    # generation_config=generation_config,
)

In [41]:
template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<|eot_id|><|start_header_id|>user<|end_header_id|>

Тебе необходимо определить учебную дисциплину в одну из категорий: математика, IT, инженерия, горное дело, менеджмент, экономика, практика, иное.

Учитывай, что в категорию математика необходимо включать все математические и физические дисциплины.

В категорию IT необходимо включать все, что касается информатики, программирования, кибербезопасность, проектирования информационных систем, машинного обучения и тд.

В категорию инженерия необходимо включать все, что касается механизмов и материалов.

В категорию горное дело необходимо включать все, что касается производства горных работ.

В категорию менеджмент включай все, что касается управления, hr-процессов, менеджмента.

В категорию экономика все что касается экономической теории, процессов, денег.

В категорию практика включай все что касается учебной, производственной, преддипломной практике, научно-исследовательские работы.

В категорию иное включай все что касается истории, философии, изучения иностранных языков, психологии, права, физической культуры и тп.

Если ни одна из категорий не подходит, то ответить "я не знаю" (без кавычек). Отвечай ТОЛЬКО одной из категорий или "я не знаю", НЕ пиши, почему ты принял такое решение.

За успешной выполнение задачи тебе заплатят 10 долларов.

Пример1
Дисциплина: История
Ответ: иное

Пример2
Дисциплина: Программирование и алгоритмизация
Ответ: IT

Пример3
Дисциплина: Основы металловедения
Ответ: инженерия

Пример4
Дисциплина: Физические свойства твердых тел
Ответ: математика

Пример5
Дисциплина: Сипушатник
Ответ: я не знаю



Дисциплина: {cource}
Ответ:<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [42]:
prompt = PromptTemplate(template=template, input_variables=["cource"])
sent = prompt.format(cource="Математический анализ")
print(text_pipeline([sent])[0][0]["generated_text"])


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<|eot_id|><|start_header_id|>user<|end_header_id|>

Тебе необходимо определить учебную дисциплину в одну из категорий: математика, IT, инженерия, горное дело, менеджмент, экономика, практика, иное.

Учитывай, что в категорию математика необходимо включать все математические и физические дисциплины.

В категорию IT необходимо включать все, что касается информатики, программирования, кибербезопасность, проектирования информационных систем, машинного обучения и тд.

В категорию инженерия необходимо включать все, что касается механизмов и материалов.

В категорию горное дело необходимо включать все, что касается производства горных работ.

В категорию менеджмент включай все, что касается управления, hr-процессов, менеджмента.

В категорию экономика все что касается экономической теории, процессов, денег.

В категорию практика включай вс

In [43]:
print(df.dis.nunique())
ds = pd.DataFrame({"col": df.dis.unique()})
ds = Dataset.from_pandas(ds).with_format("torch", device=0)

1161


In [44]:
train_load = DataLoader(ds, batch_size=8, num_workers=0)
res = []
for batch in tqdm(train_load):
    with torch.no_grad():
        sents = [prompt.format(cource=i) for i in batch["col"]]
        len_sent = [len(i) for i in sents]
        res.extend(
            [
                i[0]["generated_text"][k:].strip()
                for i, k in zip(text_pipeline(sents), len_sent)
            ]
        )

100%|██████████| 146/146 [01:48<00:00,  1.34it/s]


In [46]:
qw = pd.DataFrame({"col": df.dis.unique()})
qw["res"] = res

In [50]:
qw

,col,res
0,Инженерная компьютерная графика,инженерия
1,Иностранный язык,иное
2,История,иное
3,Математика,математика
4,Программирование и алгоритмизация,IT
...,...,...
1156,Теория вероятностей и математическая статистика для инженеров,математика
1157,Практическое применение машинного обучения,IT
1158,Язык SQL,IT
1159,Глубокое обучение,IT


In [49]:
qw.res.value_counts()

res
IT                                248
инженерия                         236
математика                        146
горное дело                       141
менеджмент                        137
иное                              125
практика                           69
экономика                          51
Экономика                           2
экономика, менеджмент               1
информатика                         1
иностранные языки                   1
информационные технологии (IT)      1
информационные технологии           1
физическая культура, иное           1
Name: count, dtype: int64

In [51]:
qw["res"] = qw["res"].replace(
    {
        "Экономика": "экономика",
        "экономика, менеджмент": "экономика",
        "информатика": "IT",
        "иностранные языки": "иное",
        "информационные технологии (IT)": "IT",
        "информационные технологии": "IT",
        "физическая культура, иное": "иное",
    }
)

In [52]:
qw.res.value_counts()

res
IT             251
инженерия      236
математика     146
горное дело    141
менеджмент     137
иное           127
практика        69
экономика       54
Name: count, dtype: int64

In [55]:
df.sample()

,hash,level,group,spec,year,half_year,dis,evaluation_1,evaluation_2
328551,197b83f9730b57bce23b2ee510d1d093,Бакалавр,БПИ-21-4,Прикладная информатика,2022 - 2023,II полугодие,Разработка клиент-серверных приложений,Отлично,Отлично


In [56]:
qw.columns = ["dis", "dis_group"]

In [59]:
df = df.merge(qw, on="dis")

In [60]:
df.sample(3)

,hash,level,group,spec,year,half_year,dis,evaluation_1,evaluation_2,dis_group
106185,2ed17018b526152df15b88da8e41dfb7,Бакалавр,БМН-19-1з,Менеджмент,2020 - 2021,II полугодие,Методы оптимальных управленческих решений,NaN,Отлично,менеджмент
195323,bb167e1db4feef55d83b6ea51eaf2328,Бакалавр,БЛГ-20-5,Лингвистика,2021 - 2022,I полугодие,Основы публичной речи,NaN,NaN,менеджмент
213815,8325f664300dcd28750130b020a9d22d,Бакалавр,БЛГ-20-15,Лингвистика,2022 - 2023,I полугодие,Сравнительная типология английского и русского языков,NaN,NaN,иное


Посчитаем какой год зачисления и курс на котором находился студент на момент сдачи экзамена / зачета

In [61]:
df["year_start"] = df["group"].apply(lambda X: X.split("-")[1])

In [62]:
df["year_start"].value_counts()

year_start
20    95942
21    53926
22    42061
19    13130
18    11236
17      313
16       77
Name: count, dtype: int64

In [63]:
def course(x1, x2):
    return int(x2.split()[-1][-2:]) - int(x1)

In [64]:
df["course"] = df.apply(lambda X: course(X["year_start"], X["year"]), axis=1)

In [65]:
df[["level", "course"]].value_counts().sort_index()

level       course
Бакалавр    1         68349
            2         62559
            3         33836
            4           537
Магистр     1         13744
Специалист  1         11697
            2         10758
            3          7778
            4          5087
            5          2256
            6            73
            7            11
Name: count, dtype: int64

У Бакалавров есть 5 курс, а у специалиста 6, 7, 8. Их немного поэтому просто удалим такие значения.

In [66]:
bool_bad_bac = df["level"].isin(["Бакалавр"]) & df["course"].isin([5])
bool_bad_mag = df["level"].isin(["Специалист"]) & df["course"].isin([6, 7, 8])

df = df[(~bool_bad_bac) & (~bool_bad_mag)]
df.shape

(216601, 12)

In [67]:
df[["level", "course"]].value_counts().sort_index()

level       course
Бакалавр    1         68349
            2         62559
            3         33836
            4           537
Магистр     1         13744
Специалист  1         11697
            2         10758
            3          7778
            4          5087
            5          2256
Name: count, dtype: int64

Удалим 20% null значений c 3 курса Бакалавра и Специалистов 

In [68]:
bool_values = (
    df["evaluation_1"].isnull()
    & df["evaluation_2"].isnull()
    & (
        (df["level"].isin(["Бакалавр", "Специалист"]) & (df["course"] > 2))
        # | df["level"].isin(["Магистр"])
    )
)


df_new = pd.concat(
    [df[bool_values].groupby("hash").sample(frac=0.8, random_state=1), df[~bool_values]]
)

In [69]:
df_new.evaluation_2.value_counts()

evaluation_2
Отлично                52823
Удовлетворительно      42286
Хорошо                 39680
зачтено                39309
Неявка                  3873
Неудовлетворительно     1146
не зачтено               896
Не допущен                 1
Name: count, dtype: int64

In [70]:
df_new[["evaluation_1", "evaluation_2"]] = df_new[
    ["evaluation_1", "evaluation_2"]
].fillna("Неудовлетворительно")

In [71]:
df_new[["evaluation_1", "evaluation_2"]] = df_new[
    ["evaluation_1", "evaluation_2"]
].replace(
    [
        "Неудовлетворительно",
        "Неявка",
        "Не допущен",
        "Удовлетворительно",
        "Хорошо",
        "Отлично",
        "зачтено",
        "не зачтено",
    ],
    [2, 2, 2, 3, 4, 5, "credit", "not_credit"],
)

In [72]:
df_new.sample(5)

,hash,level,group,spec,year,half_year,dis,evaluation_1,evaluation_2,dis_group,year_start,course
13617,1468ec5d00fc17883056cff268395577,Бакалавр,БЛГ-20-11,Лингвистика,2021 - 2022,II полугодие,Практическая грамматика,2,3,практика,20,2
142685,8e174a7b45ceea5a6a653bd54758998c,Бакалавр,БМТ-21-1,Металлургия,2022 - 2023,II полугодие,Математика,2,3,математика,21,2
53605,3ee5c1f2cb146a8aa99fd014954d5aed,Специалист,СГД-21-2,Горное дело,2021 - 2022,II полугодие,Информатика,not_credit,credit,IT,21,1
37258,cd2b73b2b7fab90c695b3995e4f97f90,Бакалавр,БПМ-20-2,Прикладная математика,2022 - 2023,I полугодие,Дискретные и нелинейные системы автоматического управления,5,5,математика,20,3
104763,fe91dc70c56f0d5095a4c98771a8a9b7,Бакалавр,БЛГ-22-13,Лингвистика,2022 - 2023,II полугодие,История,credit,credit,иное,22,1


In [73]:
df_new["time_period"] = df_new["year"] + "_" + df_new["half_year"]

In [75]:
time_period = []
yaer_list = [
    "2018 - 2019",
    "2019 - 2020",
    "2020 - 2021",
    "2021 - 2022",
    "2022 - 2023",
    "2023 - 2024",
]
half_yaer_list = ["I полугодие", "II полугодие"]
result = {}

for year in tqdm(yaer_list):
    for half_year in half_yaer_list:
        time_period.append(f"{year}_{half_year}")

        df_buff = df_new[df_new["time_period"].isin(time_period)]
        df_buff_exam = df_buff[df_buff.evaluation_2.isin([2, 3, 4, 5])]
        df_buff_credit = df_buff[~df_buff.evaluation_2.isin([2, 3, 4, 5])]

        if not len(df_buff):
            continue

        result[time_period[-1]] = {}

        for i in ["hash", "level", "group", "spec", "dis_group"]:
            gb_data = df_buff_exam.groupby(i)["evaluation_2"].agg(
                mean="mean", median="median", std="std"
            )

            df_pivot = df_buff_credit[[i, "evaluation_2", "time_period"]].pivot_table(
                index=i, columns="evaluation_2", aggfunc="count"
            )
            df_pivot.columns = [i[1] for i in df_pivot.columns]

            if "не зачтено" not in df_pivot.columns:
                df_pivot["not_credit"] = 0
            if "зачтено" not in df_pivot.columns:
                df_pivot["credit"] = 0

            gb_data = gb_data.join(df_pivot)

            gb_data["credit_part"] = gb_data["not_credit"] / (
                gb_data["credit"] + gb_data["not_credit"]
            )
            gb_data.columns = [f"{i}_{col}" for col in gb_data.columns]

            result[time_period[-1]][i] = gb_data.fillna(0).reset_index()

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:12<00:00,  2.09s/it]


In [76]:
index = 0
res_df = []
train_data = df_new[
    ["hash", "level", "group", "spec", "dis", "dis_group", "time_period", "course", "evaluation_2"]
]

for period in tqdm(time_period[1:]):

    df_merge = train_data[train_data["time_period"] == period]
    dict_merge = result[time_period[index]]

    for col_name in dict_merge.keys():
        df_merge = pd.merge(
            df_merge, dict_merge[col_name], on=col_name, how="left"
        ).fillna(0)

    res_df.append(df_merge)
    index += 1

100%|██████████| 11/11 [00:01<00:00,  7.02it/s]


In [78]:
df_train = pd.concat(res_df)

In [79]:
df_train.shape

(213564, 39)

In [100]:
df_train.columns

Index(['hash', 'level', 'group', 'spec', 'dis', 'dis_group', 'time_period', 'course', 'evaluation_2', 'hash_mean', 'hash_median', 'hash_std', 'hash_credit', 'hash_not_credit', 'hash_credit_part', 'level_mean', 'level_median', 'level_std', 'level_credit', 'level_not_credit', 'level_credit_part', 'group_mean', 'group_median', 'group_std', 'group_credit', 'group_not_credit', 'group_credit_part', 'spec_mean', 'spec_median', 'spec_std', 'spec_credit', 'spec_not_credit', 'spec_credit_part', 'dis_group_mean', 'dis_group_median', 'dis_group_std', 'dis_group_credit', 'dis_group_not_credit', 'dis_group_credit_part'], dtype='object')

Признаки 'hash_mean', 'hash_median', 'hash_std', 'hash_credit', 'hash_not_credit', 'hash_credit_part', 'level_mean', 'level_median', 'level_std', 'level_credit', 'level_not_credit', 'level_credit_part', 'group_mean', 'group_median', 'group_std', 'group_credit', 'group_not_credit', 'group_credit_part', 'spec_mean', 'spec_median', 'spec_std', 'spec_credit', 'spec_not_credit', 'spec_credit_part', 'dis_group_mean', 'dis_group_median', 'dis_group_std', 'dis_group_credit', 'dis_group_not_credit', 'dis_group_credit_part' расчитываются за прошедший период, то есть экзамен или зачет сданы в 1 полугодии 2022-2023 учебном году, то расчет их ведется без учета 1 полугодии 2022-2023. 

Считаются средняя, меданная оценка и стандартное отклонение для каждого стундента, его группы, курса, уроня образования, программы, группы предмета. 

Также для них считается сданные или не сданные зачеты, ссотношения не сданных зачетов ко всем зачетам.

### Тренировка и оценка модели

In [84]:
df_train_exam = df_train[~df_train["evaluation_2"].isin(["credit", "not_credit"])]
df_train_credit = df_train[df_train["evaluation_2"].isin(["credit", "not_credit"])]

In [85]:
df_train_exam["time_period"].value_counts(normalize=True).sort_index()

time_period
2018 - 2019_II полугодие   0.00
2019 - 2020_I полугодие    0.01
2019 - 2020_II полугодие   0.01
2020 - 2021_I полугодие    0.05
2020 - 2021_II полугодие   0.07
2021 - 2022_I полугодие    0.14
2021 - 2022_II полугодие   0.19
2022 - 2023_I полугодие    0.24
2022 - 2023_II полугодие   0.29
Name: proportion, dtype: float64

In [86]:
df_train_exam["time_period"].value_counts(normalize=True).sort_index()

time_period
2018 - 2019_II полугодие   0.00
2019 - 2020_I полугодие    0.01
2019 - 2020_II полугодие   0.01
2020 - 2021_I полугодие    0.05
2020 - 2021_II полугодие   0.07
2021 - 2022_I полугодие    0.14
2021 - 2022_II полугодие   0.19
2022 - 2023_I полугодие    0.24
2022 - 2023_II полугодие   0.29
Name: proportion, dtype: float64

In [87]:
df_train_exam["time_period"].unique()

array(['2018 - 2019_II полугодие', '2019 - 2020_I полугодие',
       '2019 - 2020_II полугодие', '2020 - 2021_I полугодие',
       '2020 - 2021_II полугодие', '2021 - 2022_I полугодие',
       '2021 - 2022_II полугодие', '2022 - 2023_I полугодие',
       '2022 - 2023_II полугодие'], dtype=object)

In [88]:
train_df = df_train_exam[
    ~df_train_exam.time_period.isin(
        [
            "2022 - 2023_II полугодие",
        ]
    )
]
valid_df = df_train_exam[df_train_exam.time_period.isin(["2022 - 2023_II полугодие"])]


X_train = train_df.drop(["hash", "time_period", "evaluation_2"], axis=1)
X_valid = valid_df.drop(["hash", "time_period", "evaluation_2"], axis=1)

y_train = train_df["evaluation_2"]
y_valid = valid_df["evaluation_2"]

In [89]:
model = CatBoostClassifier(
    eval_metric="AUC",
    boosting_type="Ordered",
    verbose=50,
    early_stopping_rounds=100,
    random_seed=1,
)
model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    cat_features=["level", "spec", "course", "group", "dis_group"],
    text_features=["dis"],
)

Learning rate set to 0.11888
0:	test: 0.7074891	best: 0.7074891 (0)	total: 904ms	remaining: 15m 2s
50:	test: 0.8045883	best: 0.8045883 (50)	total: 42.4s	remaining: 13m 8s
100:	test: 0.8127217	best: 0.8127217 (100)	total: 1m 22s	remaining: 12m 14s
150:	test: 0.8185840	best: 0.8185962 (149)	total: 2m 2s	remaining: 11m 30s
200:	test: 0.8220857	best: 0.8220857 (200)	total: 2m 43s	remaining: 10m 48s
250:	test: 0.8240640	best: 0.8240640 (250)	total: 3m 23s	remaining: 10m 6s
300:	test: 0.8255968	best: 0.8255968 (300)	total: 4m 2s	remaining: 9m 24s
350:	test: 0.8261966	best: 0.8262629 (341)	total: 4m 42s	remaining: 8m 42s
400:	test: 0.8273733	best: 0.8273733 (400)	total: 5m 23s	remaining: 8m 3s
450:	test: 0.8280961	best: 0.8281134 (447)	total: 6m 3s	remaining: 7m 22s
500:	test: 0.8286770	best: 0.8286770 (500)	total: 6m 44s	remaining: 6m 42s
550:	test: 0.8289167	best: 0.8289344 (526)	total: 7m 24s	remaining: 6m 2s
600:	test: 0.8289276	best: 0.8289525 (597)	total: 8m 4s	remaining: 5m 21s
650:	te

In [90]:
repor_exam = classification_report(y_valid.to_list(), model.predict(X_valid))
print(repor_exam)

              precision    recall  f1-score   support

           2       0.64      0.65      0.64     12797
           3       0.46      0.51      0.49     11035
           4       0.36      0.22      0.28     10742
           5       0.62      0.73      0.67     16512

    accuracy                           0.55     51086
   macro avg       0.52      0.53      0.52     51086
weighted avg       0.54      0.55      0.54     51086



3 и 4 разграничиваются плохо

In [91]:
# важность признаков
fi_exam = model.get_feature_importance(prettified=True)
fi_exam = fi_exam[fi_exam["Importances"] > 0]
fi_exam

,Feature Id,Importances
0,dis,26.72
1,hash_mean,14.68
2,spec_std,9.86
3,group,8.36
4,hash_std,7.68
5,spec,5.54
6,hash_median,5.51
7,dis_group,5.43
8,group_std,4.63
9,group_mean,3.19


In [92]:
df_train_credit["time_period"].value_counts(normalize=True).sort_index()

time_period
2018 - 2019_II полугодие   0.02
2019 - 2020_I полугодие    0.03
2019 - 2020_II полугодие   0.02
2020 - 2021_I полугодие    0.10
2020 - 2021_II полугодие   0.10
2021 - 2022_I полугодие    0.15
2021 - 2022_II полугодие   0.13
2022 - 2023_I полугодие    0.25
2022 - 2023_II полугодие   0.20
Name: proportion, dtype: float64

In [93]:
train_df = df_train_credit[
    ~df_train_credit.time_period.isin(
        [
            "2022 - 2023_II полугодие",
        ]
    )
]
valid_df = df_train_credit[
    df_train_credit.time_period.isin(["2022 - 2023_II полугодие"])
]


X_train = train_df.drop(["hash", "time_period", "evaluation_2"], axis=1)
X_valid = valid_df.drop(["hash", "time_period", "evaluation_2"], axis=1)

y_train = train_df["evaluation_2"]
y_valid = valid_df["evaluation_2"]

In [96]:
model = CatBoostClassifier(
    iterations=1000,
    eval_metric="AUC",
    boosting_type="Ordered",
    verbose=50,
    random_seed=1,
    early_stopping_rounds=200
)
model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    cat_features=["level", "spec", "course", "group", "dis_group"],
    text_features=["dis"],
)

Learning rate set to 0.074385
0:	test: 0.6203042	best: 0.6203042 (0)	total: 79.3ms	remaining: 1m 19s
50:	test: 0.8480276	best: 0.8481256 (42)	total: 5.6s	remaining: 1m 44s
100:	test: 0.8560049	best: 0.8576589 (72)	total: 11s	remaining: 1m 38s
150:	test: 0.8574169	best: 0.8589062 (127)	total: 16.3s	remaining: 1m 31s
200:	test: 0.8548053	best: 0.8589062 (127)	total: 22.1s	remaining: 1m 27s
250:	test: 0.8521725	best: 0.8589062 (127)	total: 27.9s	remaining: 1m 23s
300:	test: 0.8493930	best: 0.8589062 (127)	total: 33.7s	remaining: 1m 18s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8589062118
bestIteration = 127

Shrink model to first 128 iterations.


In [97]:
repor_credit = classification_report(y_valid.to_list(), model.predict(X_valid))
print(repor_credit)

              precision    recall  f1-score   support

      credit       0.94      1.00      0.97      7635
  not_credit       0.50      0.01      0.01       466

    accuracy                           0.94      8101
   macro avg       0.72      0.50      0.49      8101
weighted avg       0.92      0.94      0.92      8101



In [98]:
# важность признаков 
fi_credit = model.get_feature_importance(prettified=True)
fi_credit = fi_credit[fi_credit["Importances"] > 0]
fi_credit

,Feature Id,Importances
0,hash_mean,25.57
1,dis,11.99
2,course,10.89
3,group_std,6.85
4,hash_median,6.82
5,group,5.97
6,spec_std,5.31
7,dis_group,4.60
8,dis_group_std,4.37
9,hash_std,3.29


### Вывод

Проведена работа по очистке данных, формированию призанков, обучению и тестированию модели. 

Модель предсказания оценки за экзамен показывает умеренные результаты, для обучения предсказания зачетов требуеются больше данных или же дополнительные признаки.